In [2]:
import os
import lmdb
import tkinter as tk
from tkinter import filedialog
import glob
import cv2
from pathlib import Path

In [3]:
BASE_DIR = os.getcwd()
root = tk.Tk()
root.withdraw()

FOLDER_PATH = filedialog.askdirectory(title="Select directory containing images")
IMAGES_LIST = glob.glob(f"{FOLDER_PATH}/*.png")

In [4]:
lmdb_path = Path("/".join(str(FOLDER_PATH).split("/")[:-1]) + "/images_lmdb")
print(lmdb_path)
lmdb_path.mkdir(parents=True, exist_ok=True)
lmdb_path = lmdb_path.as_posix()

print("Generate LMDB to %s" % lmdb_path)

D:\Datasets\PVDN\images_lmdb
Generate LMDB to D:/Datasets/PVDN/images_lmdb


In [5]:
db = lmdb.open(lmdb_path, subdir=True,
                map_size=5e11, readonly=False,
                meminit=False, map_async=True, writemap=True)
    
txn = db.begin(write=True)
# for index in range(len(IMAGES_LIST)): #num_samples): (limit the number of sample to 10 vor test purpose)
for index in range(10): #num_samples): (limit the number of sample to 10 vor test purpose)
    # print(str(IMAGES_LIST[index]))
    read_img = cv2.imread(str(IMAGES_LIST[index]))
    #read_pcd = open(sample_paths[index])    
    file_path = u'{}'.format(IMAGES_LIST[index]).encode('ascii') # dumps(sample_paths[index])
    
    # TODO: finde better way to store multiple data to one key
    # write methods for the other data types (.pcd and .pcd.bin)
            
    txn.put(file_path, cv2.imencode('.jpg',read_img)[1])
    #txn.put(u'y_{}'.format(index).encode('ascii'), img_path)
    if index != 0 and index % 10 == 0:
        print("[%d]" % (index))
        txn.commit()

txn.commit()

#keys = [u'{}'.format(k).encode('ascii') for k in range(index + 1)]
#with db.begin(write=True) as txn:
#    txn.put(b'__keys__', dumps(keys))
#    txn.put(b'__len__', dumps(len(keys)))

db.sync()
db.close()
print("Done processing.")

C:\Users\u7oh2m\AppData\Local\Temp\ipykernel_24624\3010404039.py:1: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  db = lmdb.open(lmdb_path, subdir=True,


Done processing.
